In [1]:
import pandas as pd

In [10]:
final_gene = pd.read_csv('ADNI_Imaging_Gene_Expression_MMSE_geneLabeled_Combined.csv',low_memory=False)
final_gene = final_gene.T
final_gene.columns = final_gene.iloc[0]
final_gene = final_gene.drop(['PTID','Gene'])
#final_gene

In [16]:
final_gene.head()

PTID,COLPROT,RIN,VISCODE,EXAMDATE,AGE,PTGENDER,PTEDUCAT,PTETHCAT,PTRACCAT,APOE4,...,AFFX-r2-TagH_at,AFFX-r2-TagIN-3_at,AFFX-r2-TagIN-5_at,AFFX-r2-TagIN-M_at,AFFX-r2-TagJ-3_at,AFFX-r2-TagJ-5_at,AFFX-r2-TagO-3_at,AFFX-r2-TagO-5_at,AFFX-r2-TagQ-3_at,AFFX-r2-TagQ-5_at
116_S_1249,ADNIGO,7.7,m48,2/23/11,70.8,Female,15,Not Hisp/Latino,White,1,...,2.355,2.624,2.01,2.906,2.463,2.05,2.06,1.858,2.028,2.162
037_S_4410,ADNI2,7.6,bl,1/4/12,69.1,Female,14,Not Hisp/Latino,White,0,...,2.1,2.82,1.726,2.465,2.26,1.933,1.717,2.208,2.058,1.882
006_S_4153,ADNI2,7.2,bl,9/9/11,79.3,Male,20,Not Hisp/Latino,White,1,...,2.165,2.455,1.84,2.681,2.251,1.985,1.77,2.184,2.007,2.134
116_S_1232,ADNIGO,6.8,m48,2/16/11,72.1,Female,14,Not Hisp/Latino,White,0,...,2.094,2.599,1.837,2.713,2.158,1.916,1.878,2.163,2.185,2.099
099_S_4205,ADNI2,7.9,bl,11/4/11,81.4,Female,18,Not Hisp/Latino,White,0,...,1.973,2.544,1.909,2.548,2.266,2.077,1.838,2.085,1.941,1.883


In [17]:
X = final_gene.loc[:, '11715100_at':'AFFX-r2-TagQ-5_at']
y = final_gene['DX_bl']

In [18]:
from collections import Counter
Counter(y)

Counter({'CN': 260, 'AD': 43, 'EMCI': 215, 'LMCI': 226})

## Classes: CN_AD, CN_EMCI, CN_LMCI, EMCI_LMCI, EMCI_AD, LMCI_AD 

In [13]:
!pip install imbalanced-learn

     |████████████████████████████████| 206 kB 2.5 MB/s eta 0:00:01
     |████████████████████████████████| 7.2 MB 10.4 MB/s eta 0:00:01
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.23.2
    Uninstalling scikit-learn-0.23.2:
      Successfully uninstalled scikit-learn-0.23.2


In [14]:
import imblearn
print(imblearn.__version__)

0.8.0


In [43]:
from sklearn.utils import shuffle
from sklearn.preprocessing import label_binarize

classes = ['CN_AD', 'CN_EMCI', 'CN_LMCI', 'EMCI_LMCI', 'EMCI_AD', 'LMCI_AD']

df1 = final_gene[final_gene['DX_bl']=='CN']
df2 = final_gene[final_gene['DX_bl']=='AD']
df_sampled = pd.concat([df1, df2])
df_sampled = shuffle(df_sampled, random_state = 42)
y = df_sampled.DX_bl
X = df_sampled.loc[:,'11715100_at':'AFFX-r2-TagQ-5_at']
y = label_binarize(y, classes=['CN', 'AD'])
X = X.fillna(X.mean())

In [55]:
from imblearn.over_sampling import ADASYN
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(Counter(y_test.reshape(len(y_test))))
print(Counter(y_train.reshape(len(y_train))))
oversample = SMOTE(sampling_strategy=1.0, k_neighbors=7)
X_train, y_train = oversample.fit_resample(X_train, y_train)
print(Counter(y_train.reshape(len(y_train))))

Counter({0: 53, 1: 8})
Counter({0: 207, 1: 35})
Counter({0: 207, 1: 207})


In [ ]:
model = GradientBoostingClassifier(n_estimators=1000)
probas_ = model.fit(X_train, y_train).predict_proba(X_test)
acc = model.score(X_test,y_test,average='weighted')
y_pred = model.predict(X_test)
f1 = f1_score(y_test, y_pred, average='weighted')
importance =  model.feature_importances_
acc, f1

In [33]:
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from imblearn.pipeline import Pipeline
from numpy import mean
n_estimators = 1000
model = RandomForestClassifier(n_estimators=n_estimators,criterion='gini',class_weight='balanced')
# evaluate pipeline
steps = [('over', SMOTE()), ('model', model)]
pipeline = Pipeline(steps=steps)
# evaluate pipeline
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=1)
scores = cross_val_score(pipeline, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
print('Mean ROC AUC: %.3f' % mean(scores))

Mean ROC AUC: 0.605
